# Collection of Corona Infection Status (using OpenAPI)

In [14]:
import urllib.request
from bs4 import BeautifulSoup # HTML, XML Document 데이터를 읽기 위한 라이브러리
import pandas as pd
import datetime as dt


def strptime_statedt(x):
    return dt.datetime.strptime(x, '%Y%m%d')

serviceKey = '0Nxjl3xb59fkPcq7n0%2FoeaP1%2BvZy8L1l%2B4NunrGvuPrFcORrGfI2caBJ2dzsJA6VZWYw6XGIJ0hH2%2Bx9V74bUQ%3D%3D'
startCreateDt = '20200214'
endCreateDt = '20201214'
url_xml = str('http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson?serviceKey=' + serviceKey + 
              '&startCreateDt=' + startCreateDt + '&endCreateDt=' + endCreateDt)

response = urllib.request.urlopen(url_xml)
response_code = response.getcode()


corona_df = pd.DataFrame({'statedt': [], 'createdt': [], 'decidecnt': [], 'examcnt':[], 'clearcnt':[], 'deathcnt':[]})

if response_code == 200:
    response_body = response.read().decode('utf-8') # decode('utf-8') : 바이트열 -> 문자열(복호화)
    soup = BeautifulSoup(response_body, 'html.parser') # xml도 가능
    dataList = soup.find_all('item')

    for data in dataList:
        # 마지막행에 데이터 추가 (모두 문자열로 받아옴)
        corona_df.loc[len(corona_df.index)] = [data.statedt.text, data.createdt.text, data.decidecnt.text, data.examcnt.text, data.clearcnt.text, data.deathcnt.text]
        
    ##### 데이터프레임 정렬 #####
    corona_df['statedt'] = corona_df['statedt'].apply(strptime_statedt) # statedt기준 정렬을 위해 날짜형식으로 변환
    corona_df['createdt'] = pd.to_datetime(corona_df['createdt']) # 날짜형식으로 변환
    corona_df['decidecnt'] = corona_df['decidecnt'].astype('int') # 정수형으로 변환
    
    # statedt(기준일) 기준으로는 오름차순, createdt(등록일시분초) 기준으로는 내림차순 정렬
    corona_df = corona_df.sort_values(by=["statedt", "createdt"], ascending=[True, False])
    
    # statedt열에서 중복행 제거(처음 나타난 데이터는 보존 = 그 날의 가장최신 데이터)
    corona_df = corona_df.drop_duplicates(subset=['statedt'])
    
    
    ##### 일별 확진자수 #####
    corona_df['shift_down'] = corona_df['decidecnt'].shift(+1).fillna(0)
    corona_df['decidecnt_day'] = corona_df['decidecnt'] - corona_df['shift_down']
    corona_df['decidecnt_day'] = corona_df['decidecnt_day'].astype('int')
    corona_df.drop(corona_df.index[0], inplace=True) # 첫번째 행 제거(일별확진자 수 계산하는과정에서 첫행은 누적값 그대로 들어가기때문)
    
print(corona_df.info())
display(corona_df)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 304 entries, 327 to 0
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   statedt        304 non-null    datetime64[ns]
 1   createdt       304 non-null    datetime64[ns]
 2   decidecnt      304 non-null    int32         
 3   examcnt        304 non-null    object        
 4   clearcnt       304 non-null    object        
 5   deathcnt       304 non-null    object        
 6   shift_down     304 non-null    float64       
 7   decidecnt_day  304 non-null    int32         
dtypes: datetime64[ns](2), float64(1), int32(2), object(3)
memory usage: 19.0+ KB
None


,statedt,createdt,decidecnt,examcnt,clearcnt,deathcnt,shift_down,decidecnt_day
327,2020-02-15,2020-02-15 17:00:21.210,28,558,9,0,28.0,0
325,2020-02-16,2020-02-16 16:57:02.020,29,485,9,0,28.0,1
323,2020-02-17,2020-02-17 16:50:47.470,30,708,10,0,29.0,1
320,2020-02-18,2020-02-18 16:42:00.000,31,818,12,0,30.0,1
318,2020-02-19,2020-02-19 17:00:13.130,51,1149,16,0,31.0,20
...,...,...,...,...,...,...,...,...
4,2020-12-10,2020-12-10 09:27:13.236,40098,72764,30637,564,39432.0,666
3,2020-12-11,2020-12-11 09:33:06.394,40786,77688,31157,572,40098.0,688
2,2020-12-12,2020-12-12 09:19:01.382,41736,86742,31493,578,40786.0,950
1,2020-12-13,2020-12-13 09:48:21.971,42766,90129,31814,580,41736.0,1030


# Upload Dataframe to MySQL

In [15]:
import pandas as pd
from sqlalchemy import create_engine


# Credentials to database connection
hostname = "kf99database.cu3wxbwt4src.ap-northeast-2.rds.amazonaws.com"
dbname = "kf99"
user = "admin"
pwd = "qjtwlaktmzm"

# Create dataframe
corona_status_df = corona_df.loc[:, ['statedt', 'decidecnt', 'decidecnt_day']]

# Create SQLAlchemy engine to connect to MySQL Database
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=hostname, db=dbname, user=user, pw=pwd))

# Convert dataframe to sql table                                   
corona_status_df.to_sql(name='corona_status', con=engine, index=False) # 새로 'table' 테이블을 만들어서 알아서 데이터프레임 생성(이미 있으면 불가)

# Define a function to collect daily data

In [16]:
import datetime
import pandas as pd
from sqlalchemy import create_engine


def strptime_statedt(x):
    return dt.datetime.strptime(x, '%Y%m%d')


def insert_dataframe_to_MySQL(corona_df):
    # Credentials to database connection
    hostname = "kf99database.cu3wxbwt4src.ap-northeast-2.rds.amazonaws.com"
    dbname = "kf99"
    user = "admin"
    pwd = "qjtwlaktmzm"

    # Create dataframe
    corona_status_df = corona_df.loc[:, ['statedt', 'decidecnt', 'decidecnt_day']]

    # Create SQLAlchemy engine to connect to MySQL Database
    engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=hostname, db=dbname, user=user, pw=pwd))

    # Convert dataframe to sql table                                   
    corona_status_df.to_sql(name='corona_status', con=engine, index=False, if_exists='append')

    return None



# 당일 데이터 생성시간이 주로 9시30분 근처이므로 매일 10시에 함수를 자동호출하도록 하는 것이 좋겠다
def update_daily_corona_status():
    # datetime.strftime() 메서드는 datetime 객체를 다양한 포맷의 문자열로 변환
    # datetime.strptime() 메서드는 다양한 포맷의 문자열을 datetime 객체로 변환
    now = datetime.datetime.now() # 2020-12-14 20:59:02.913798
    today = now.strftime('%Y%m%d') # 20201214 (str)

    oneday = datetime.timedelta(days=1)
    yesterday = (now - oneday).strftime('%Y%m%d') # 20201213 (str)
    
    
    ##### collect the corona data using OpenAPI #####
    serviceKey = '0Nxjl3xb59fkPcq7n0%2FoeaP1%2BvZy8L1l%2B4NunrGvuPrFcORrGfI2caBJ2dzsJA6VZWYw6XGIJ0hH2%2Bx9V74bUQ%3D%3D'
    startCreateDt = yesterday
    endCreateDt = today
    url_xml = str('http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson?serviceKey=' + serviceKey + 
                  '&startCreateDt=' + startCreateDt + '&endCreateDt=' + endCreateDt)
    
    response = urllib.request.urlopen(url_xml)
    response_code = response.getcode()
    
    corona_df = pd.DataFrame({'statedt': [], 'createdt': [], 'decidecnt': []})

    if response_code == 200:
        response_body = response.read().decode('utf-8') # decode('utf-8') : 바이트열 -> 문자열(복호화)
        soup = BeautifulSoup(response_body, 'html.parser') # xml도 가능
        dataList = soup.find_all('item')
        
        for data in dataList:
            # 마지막행에 데이터 추가 (모두 문자열로 받아옴)
            corona_df.loc[len(corona_df.index)] = [data.statedt.text, data.createdt.text, data.decidecnt.text]

        ##### 데이터프레임 정렬 #####
        corona_df['statedt'] = corona_df['statedt'].apply(strptime_statedt) # statedt기준 정렬을 위해 날짜형식으로 변환
        corona_df['createdt'] = pd.to_datetime(corona_df['createdt']) # 날짜형식으로 변환
        corona_df['decidecnt'] = corona_df['decidecnt'].astype('int') # 정수형으로 변환

        # statedt(기준일) 기준으로는 오름차순, createdt(등록일시분초) 기준으로는 내림차순 정렬
        corona_df = corona_df.sort_values(by=["statedt", "createdt"], ascending=[True, False])

        # statedt열에서 중복행 제거(처음 나타난 데이터는 보존 = 그 날의 가장최신 데이터)
        corona_df = corona_df.drop_duplicates(subset=['statedt'])


        ##### 일별 확진자수 #####
        corona_df['shift_down'] = corona_df['decidecnt'].shift(+1).fillna(0)
        corona_df['decidecnt_day'] = corona_df['decidecnt'] - corona_df['shift_down']
        corona_df['decidecnt_day'] = corona_df['decidecnt_day'].astype('int')
        corona_df.drop(corona_df.index[0], inplace=True) # 첫번째 행 제거(일별확진자 수 계산하는과정에서 첫행은 누적값 그대로 들어가기때문)

        
    insert_dataframe_to_MySQL(corona_df)
        
    return None


update_daily_corona_status()

# Update one row, each values

In [16]:
import pymysql
import sys
import json
import time

REGION = 'ap-northeast-2'
rds_host  = "kf99database.cu3wxbwt4src.ap-northeast-2.rds.amazonaws.com"
name = "admin"
password = "qjtwlaktmzm"
db_name = "kf99"
conn = pymysql.connect(rds_host, user=name, passwd=password, db=db_name, connect_timeout=5)

def insert_data():
    with conn.cursor() as cur:
        # 테이블 test로 만들었다고 가정, 컬럼은 statedt, decidecnt로 가정
        statedt = '2020-12-15'
        decidecnt = 5000
        decidecnt_day = 100
        cur.execute("insert into test2(statedt, decidecnt, decidecnt_day) values(" + statedt + "," + str(decidecnt) + "," + str(decidecnt_day) + ")")
        conn.commit()
        cur.close()
        
insert_data()